In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
import sys
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
class_name = 'chair'
do_mmd = True
save_res = True
top_out_dir = top_evaluation_dir

sample_mmd = False
n_samples = 5
ref_pop = 1000
sample_pop = 1000

mmd_loss = 'chamfer'
boost_samples = 5
special_tag = 'coverage_chamfer_boost_5'
skip = ['test', 'train']

In [10]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()

In [11]:
best_by_jsd = {'gmm': 'bneck_128_full_32_gaussians',
               'l_w_gan_large': 1800,
               'r_gan': 1700,
               'l_gan_chamfer': 300,
               'l_gan_emd': 100,
               'l_w_gan_small': 1800
              }

best_by_mmd = {'gmm': 'bneck_128_full_40_gaussians',               
               'l_w_gan_large': 2000,
               'r_gan': 1350,
               'l_gan_chamfer': 300,
               'l_gan_emd': 200,
               'l_w_gan_small': 1700
              }

In [12]:
generators_zoo = stored_synthetic_samples(class_name).keys()

In [15]:
for best_model, tag in zip([best_by_jsd, best_by_mmd], ['jsd', 'mmd']):
    print tag    
    for experiment_name in generators_zoo:
        print experiment_name
        top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]        
        if save_res:
            exp_name = osp.basename(top_sample_dir)
            f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + '_'+ tag + '_' + exp_name + '.txt'), 'w')
        else:            
            f_mmd = sys.stdout

        sample_files = []

        if 'gan' in experiment_name:
            epoch = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
        elif 'gmm' in experiment_name:
            gaussian_type = best_model[experiment_name]
            sample_files.append(osp.join(top_sample_dir, gaussian_type + '.npz'))
        else:
            assert False
                
        for sample_file in sample_files:
            evaluator.prepare_sample_data(sample_file, boost_samples)
            sample_name = osp.basename(sample_file)[:-len('.npz')]
            print sample_name

        f_mmd.write(sample_name + '\n')

        evaluator.compute_coverage(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                              ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                              f_out=f_mmd, skip=skip)

        f_mmd.flush()
        print
        if save_res:
            f_mmd.close()

jsd
l_w_gan_large
epoch_1800
val 0.733727810651 0.0

gmm
bneck_128_full_32_gaussians
val 0.781065088757 0.0

r_gan
epoch_1700
val 0.559171597633 0.0

l_gan_chamfer
epoch_300
val 0.671597633136 0.0

l_w_gan_small
epoch_1800
val 0.727810650888 0.0

l_gan_emd
epoch_100
val 0.62426035503 0.0

mmd
l_w_gan_large
epoch_2000
val 0.748520710059 0.0

gmm
bneck_128_full_40_gaussians
val 0.789940828402 0.0

r_gan
epoch_1350
val 0.609467455621 0.0

l_gan_chamfer
epoch_300
val 0.668639053254 0.0

l_w_gan_small
epoch_1700
val 0.713017751479 0.0

l_gan_emd
epoch_200
val 0.647928994083 0.0

